In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import pandas as pd

In [5]:
def setup_driver():
    # Get the path for the ChromeDriver
    driver_path = ChromeDriverManager().install()

    # Set up Chrome options or capabilities (if needed)
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Create a Chrome service with the driver path
    chrome_service = Service(driver_path)

    # Initialize the Chrome WebDriver with options and service
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    return driver

In [50]:
def scrape_menu_item(driver, url, restaurant_id):
    try:
        driver.get(url)
        # Wait for the page to load completely (optional, but recommended)
        driver.implicitly_wait(10)  # Wait for up to 10 seconds for elements to appear

        # Get the store name
        wait = WebDriverWait(driver, 10)
        store_name_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'StoreAuthHeader__StoreName-ihWYeu')))
        # Get the text of the store name element
        store_name = store_name_element.text
        print(store_name)

        # find sections that contain the cat btn
        cat_sections = driver.find_elements(By.CSS_SELECTOR, ".OrderingMenuCategoryChip__Wrapper-jaTTMW.dfpPQD")
        cat_btn_lst = [cat_section.find_element(By.CSS_SELECTOR, """div[role="button"]""") for cat_section in cat_sections]

        item_list = []
        # iterate through cat btns, click the btn then find ele
        processed_categories = set()  # Keep track of processed categories
        for btn in cat_btn_lst:
            btn.click()
            time.sleep(1)  # Wait for the page to scroll and load items

            # Find all categories in the current section
            cats = driver.find_elements(By.CSS_SELECTOR, 'div[data-index]')

            for cat in cats:
                cat_element = cat.find_element(By.TAG_NAME, 'h4')
                cat_name = cat_element.text

                if cat_name not in processed_categories:
                    processed_categories.add(cat_name)

                    # Find all items within the current category
                    menu_items = cat.find_elements(By.CSS_SELECTOR, "li[data-test-id='menuItem']")
                    for item in menu_items:
                        # print(item.text)
                        item_info = item.text.split('\n')
                        if len(item_info) == 3:
                            item_list.append([cat_name, item_info[0], item_info[1], item_info[2]])
                        elif len(item_info) == 2:
                            item_list.append([cat_name, item_info[0], None, item_info[1]])
                        else:
                            item_list.append([None, None, None, None])
                        # item_name = item.find_element(By.CSS_SELECTOR, "div[data-test-id='menuItem-name']").text
                        
                        # try:
                        #     item_description = item.find_element(By.CSS_SELECTOR, "div[data-test-id='menuItem-description']").text
                        # except:
                        #     item_description = None

                        # item_price = item.find_element(By.CSS_SELECTOR, "div[data-test-id='menuItem-price']").text
                        # # Print or store the extracted data as needed
                        # print(cat_name, item_name, item_description, item_price)
        print(store_name, item_list)
        return store_name, item_list, restaurant_id
    except:
        return None, None, restaurant_id

In [ ]:
# def scrape_menu_item(driver, url):
#     driver.get(url)
#     # Wait for the page to load completely (optional, but recommended)
#     driver.implicitly_wait(10)  # Wait for up to 10 seconds for elements to appear

#     # Get the store name
#     wait = WebDriverWait(driver, 10)
#     store_name_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'StoreAuthHeader__StoreName-ihWYeu')))
#     # Get the text of the store name element
#     store_name = store_name_element.text
#     print(store_name)

#     # Lists to store item details
#     item_names = []
#     item_descriptions = []
#     item_prices = []
#     item_category = []

#     # Set to record found categories
#     found_categories = set()

#     # Scroll down and search for new categories
#     last_height = 0
#     while True:
#         # Get the current height of the page
#         current_height = driver.execute_script("return document.body.scrollHeight")
        
#         # Scroll down by 200 pixels
#         driver.execute_script("window.scrollTo(0, arguments[0]);", last_height + 200)
#         time.sleep(1)  # Wait for new content to load
        
#         # Update the last height
#         last_height = current_height

#         # Search for new categories and process them
#         categories = driver.find_elements(By.CSS_SELECTOR, 'div[data-index]')
#         for category in categories:
#             category_element = category.find_element(By.CLASS_NAME, 'MuiListSubheader-root')
#             category_name = category_element.text

#             if category_name not in found_categories:
#                 found_categories.add(category_name)  # Add the category to the set of found categories

#                 print("Category Name:", category_name)

#                 # Find all items within the current category
#                 menu_items = category.find_elements(By.CSS_SELECTOR, "li[data-test-id='menuItem']")
                
#                 # Iterate through each item and print its details
#                 for item in menu_items:
#                     item_name = item.find_element(By.CSS_SELECTOR, "[data-test-id='menuItem-name']").text
#                     try:
#                         item_description = item.find_element(By.CSS_SELECTOR, "[data-test-id='menuItem-description']").text
#                     except:
#                         item_description = None
#                     item_price = item.find_element(By.CSS_SELECTOR, "[data-test-id='menuItem-price']").text

#                     item_category.append(category_name)
#                     item_names.append(item_name)
#                     item_descriptions.append(item_description)
#                     item_prices.append(item_price)

#                     print(category_name, item_name, item_prices)

#                 print()  # Add a newline for better readability between categories
        
#         # Check if we've reached the bottom of the page
#         if current_height == last_height:
#             break
    
#     return store_name, item_category, item_names, item_descriptions, item_prices

In [7]:
# def scrape_menu_item(driver, url):
#     driver.get(url)

#     # Wait for the page to load completely (optional, but recommended)
#     driver.implicitly_wait(10)  # Wait for up to 10 seconds for elements to appear

#     # Get the store name
#     wait = WebDriverWait(driver, 10)
#     store_name_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'StoreAuthHeader__StoreName-ihWYeu')))

#     # Get the text of the store name element
#     store_name = store_name_element.text
#     print(store_name)

#     # Lists to store item details
#     item_names = []
#     item_descriptions = []
#     item_prices = []
#     item_category = []

#     # Set to record found categories
#     found_categories = set()

#     # Process initial categories on the page
#     categories = driver.find_elements(By.CSS_SELECTOR, 'div[data-index]')

#     for category in categories:
#         category_element = category.find_element(By.TAG_NAME, 'h4')
#         category_name = category_element.text

#         if category_name not in found_categories:
#             found_categories.add(category_name)  # Add the category to the set of found categories
#             print("Category Name:", category_name)

#             # Find all items within the current category
#             menu_items = category.find_elements(By.CSS_SELECTOR, "li[data-test-id='menuItem']")

#             # Iterate through each item and print its details
#             for item in menu_items:
#                 # item name
#                 item_name = item.find_element(By.CSS_SELECTOR, "[data-test-id='menuItem-name']").text

#                 # item description
#                 try:
#                     item_description = item.find_element(By.CSS_SELECTOR, "[data-test-id='menuItem-description']").text
#                 except:
#                     item_description = None

#                 # item price
#                 item_price = item.find_element(By.CSS_SELECTOR, "[data-test-id='menuItem-price']").text

#                 item_category.append(category_name)
#                 item_names.append(item_name)
#                 item_descriptions.append(item_description)
#                 item_prices.append(item_price)

#                 print(category_name, item_name, item_price)

#             print()  # Add a newline for better readability between categories

#     # Scroll down to the bottom and scrape one more time
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(2)  # Wait for new content to load

#     # Search for new categories and process them
#     categories = driver.find_elements(By.CSS_SELECTOR, 'div[data-index]')

#     for category in categories:
#         category_element = category.find_element(By.TAG_NAME, 'h4')
#         category_name = category_element.text

#         if category_name not in found_categories:
#             found_categories.add(category_name)  # Add the category to the set of found categories
#             print("Category Name:", category_name)

#             # Find all items within the current category
#             menu_items = category.find_elements(By.CSS_SELECTOR, "li[data-test-id='menuItem']")

#             # Iterate through each item and print its details
#             for item in menu_items:
#                 # item name
#                 item_name = item.find_element(By.CSS_SELECTOR, "[data-test-id='menuItem-name']").text

#                 # item description
#                 try:
#                     item_description = item.find_element(By.CSS_SELECTOR, "[data-test-id='menuItem-description']").text
#                 except:
#                     item_description = None

#                 # item price
#                 item_price = item.find_element(By.CSS_SELECTOR, "[data-test-id='menuItem-price']").text

#                 item_category.append(category_name)
#                 item_names.append(item_name)
#                 item_descriptions.append(item_description)
#                 item_prices.append(item_price)

#                 print(category_name, item_name, item_price)

#             print()  # Add a newline for better readability between categories

#     return store_name, item_category, item_names, item_descriptions, item_prices

In [36]:
# ===Read Restaurant Link===
df = pd.read_json("restaurant_final.json")

# ===Create DF store menu info===
df2 = pd.DataFrame()

In [52]:
driver_path = ChromeDriverManager().install()
# Initialize the Chrome driver
driver = setup_driver()

store_name_list, item_list, id_list = [], [], []
for a_store in range(101, df.shape[0]):
    print(f"The {a_store} store")
    # url = 'https://shop.ichefpos.com/store/CylxiF4S/ordering?utm_source=safer_takeout&utm_medium=website&utm_campaign=safer_takeout&utm_id=safer_takeout_01'
    url, restaurant_id = df['link'][a_store], df['id'][a_store]
    store_name, items, id = scrape_menu_item(driver, url, restaurant_id)
    store_name_list.append(store_name)
    item_list.append(items)
    id_list.append(id)

# Close the browser
driver.quit()

# Create a new DataFrame to store the scraped data
scraped_data = pd.DataFrame({
    'Store_name': store_name_list,
    'Items': item_list,
    'id': id_list
})

# Merge the scraped data with the original DataFrame df based on index or any other common column
df2 = pd.concat([df2, scraped_data], axis=0)


The 101 store
The 102 store
朋來客棧
朋來客棧 [['包裝選項', '加辣', None, 'NT$0'], ['包裝選項', '包辣', None, 'NT$0'], ['包裝選項', '袋裝', None, 'NT$0'], ['包裝選項', '盒裝', None, 'NT$0'], ['包裝選項', '不蔥蔥❌', None, 'NT$0'], ['滷味-牛肉類', '牛腱肉', None, 'NT$100'], ['滷味-牛肉類', '牛頭皮', None, 'NT$100'], ['滷味-牛肉類', '牛腸', None, 'NT$100'], ['滷味-牛肉類', '牛肚', None, 'NT$100'], ['滷味-牛肉類', '牛筋', None, 'NT$100'], ['滷味-牛肉類', '拼盤', None, 'NT$200'], ['滷味-豬肉類', '豬大腸', None, 'NT$80'], ['滷味-豬肉類', '豬小肚', None, 'NT$60'], ['滷味-豬肉類', '豬腱肉', None, 'NT$60'], ['滷味-豬肉類', '五花肉', None, 'NT$60'], ['滷味-豬肉類', '豬舌', None, 'NT$60'], ['滷味-豬肉類', '豬心', None, 'NT$60'], ['滷味-豬肉類', '豬肝', None, 'NT$60'], ['滷味-豬肉類', '肝連', None, 'NT$70'], ['滷味-豬肉類', '生腸', None, 'NT$60'], ['滷味-豬肉類', '煙燻豬頭皮', None, 'NT$40'], ['滷味-豬肉類', '煙燻豬耳朵', None, 'NT$40'], ['滷味-雞鴨類', '鴨胗', None, 'NT$40'], ['滷味-雞鴨類', '雞腸', None, 'NT$50'], ['滷味-雞鴨類', '脆腸', None, 'NT$60'], ['滷味-雞鴨類', '雞翅', None, 'NT$30'], ['滷味-雞鴨類', '鴨翅', None, 'NT$35'], ['滷味-雞鴨類', '鴨頭', None, 'NT$55'], ['滷味-雞鴨類', '煙燻雞腿', None, 'NT$70'

In [51]:
# Close the browser
driver.quit()

scraped_data = pd.DataFrame({
    'Store_name': store_name_list,
    'Items': item_list,
    'id': id_list
})

# Merge the scraped data with the original DataFrame df based on index or any other common column
df2 = pd.concat([df2, scraped_data], axis=0)
df2

,Store_name,Items,id
0,老妹上菜 行動餐車,"[[德腸熱狗堡, A經典美式雙醬, 德腸 酸黃瓜 洋蔥 番茄醬 黃芥末醬 黑胡椒, NT$8...",restaurant-link-qoC7OL4k
1,麥町吐司工房淡水新市店,"[[滿意拼盤, 南投拼盤, 蘿蔔糕一片 荷包蛋 肉燥炒麵 附贈手作紅茶, NT$85], [...",restaurant-link-9xp07WHy
2,咘魯詩早午餐 Bruce Brunch,[],restaurant-link-mVDf1UEI
3,毛青茶室海科大門市,"[[經典茶類, 經典紅茶(中), None, NT$30], [經典茶類, 經典紅茶(大),...",restaurant-link-yBDARR35
0,炊間私房餐盒,"[[精緻餐盒便當, 大鮮綠時蔬雞腿排, 大鮮綠時蔬雞腿排 壽喜燒雞腿排 三顆菜飯糰 時令時蔬...",restaurant-link-VtzGseUu
...,...,...,...
7,拾米屋,"[[咖啡, 本日滴濾式黑咖啡【外帶 150cc】(原價$130元), None, NT$70...",restaurant-link-WUrcDYBD
8,子女居酒屋,"[[秘製冷物, 冰釀梅番茄, None, NT$90], [秘製冷物, 黃金海菜絲, Non...",restaurant-link-V7sEFIxN
9,育仁韓食館,"[[韓式飯捲, 原味飯捲, None, NT$100], [韓式飯捲, 起司飯捲, None...",restaurant-link-FQDw0lGU
10,貴婦滷味 (北投店),"[[★ 人氣商品 ★, 招牌鴨血, None, NT$50], [豆製品 / 火鍋料, 甜不...",restaurant-link-VVCEowCg


In [ ]:
88